In [1]:
# Import the dependencies.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.

coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.

for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-71.52518740809742 74.06705992527401
11.80667010965955 68.63031320222973
-15.19297626043776 132.9035433078991
-73.73510784551365 171.63182819549672
69.29937905418222 -30.9001385957784
-1.0060772322503766 101.06222832021444
55.60748802349957 -97.77205881595926
-75.88913832729504 -45.72992942460414
12.020058053488 -147.17194956981766
-58.25282943181793 20.474813253362925
36.10157406212639 -127.39836478542591
-78.048121383408 -150.70476752358613
49.09994991475051 81.38299140150741
-88.12906575358745 14.60449557288203
-12.26178437359414 73.2551665211922
-13.878782653739506 -157.86931773282456
38.976705140607436 80.70246325600277
-63.411444167831384 65.0736013312964
-87.10360429957817 -11.173283094591625
20.83807610775382 -61.552221018393354
-88.13890989895633 -22.500769153504336
8.373724987834848 -109.23991286520597
-51.051017475795184 -18.643569704688787
-6.413376159607722 49.54043636522701
33.43342604261507 45.37923335613186
-27.347470836340015 156.85484574104123
30.673049908511445 112.4

-84.4148885626124 -26.60882625976035
-30.070755139513473 -155.93463076746863
12.9880990886506 121.6656782097782
34.33289405060367 9.659515182914816
46.573960147150586 46.34942236030565
-21.1770662832881 66.940000272719
67.24563771381457 -45.7041461678385
13.70117821831262 28.466761197471016
-1.0432365431067439 119.42069421497524
-66.92768609540519 -34.34514413955088
-34.40105269812851 -1.854940203298895
-85.16753494344961 -174.80485401249177
-63.735451177415904 142.87981386205473
-14.15582342580413 -35.266826415648694
73.42906554068799 -148.10095327429917
3.680104207941241 -0.28030618896545434
-4.231070155763419 66.1113021797538
-84.72801093682926 168.67715153734355
42.19636161989985 109.51963091195006
13.461933784394688 24.703286411684417
-33.09835440168521 53.28983971272427
39.24453874049476 107.8513851012948
-26.04794528356404 106.16110439733109
26.28045493243576 -136.62085949727026
51.29526154093901 -99.63012812779695
74.63660485141875 -94.26130936316247
50.308185363068475 -171.454

In [5]:
from citipy import citipy

In [6]:
# Create a list for holding the cities.

cities = []

# Identify the nearest city for each latitude and longitude combination.

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.

len(cities)

814

In [7]:
# Import the time library and the datetime module from the datetime library 

import time
from datetime import datetime

In [8]:
# Import the requests library.

import requests

# Import the API key.

from config import weather_api_key

In [9]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=b61e5efacc42e3559cf36db1e366e2cd


In [36]:
# Create an empty list to hold the weather data.

city_data = []

# Print the beginning of the logging.

print("Beginning Data Retrieval     ")

print("-----------------------------")

# Create counters.

record_count = 1
set_count = 1

# Loop through all the cities in the list.

for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    
    record_count += 1
    
    # Run an API request for each of the cities.
    
    try:
        # Parse the JSON and retrieve data.
        
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard.
        
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_desc})

# If an error is experienced, skip the city.

    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.

print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | saint-philippe
Processing Record 2 of Set 1 | kavaratti
Processing Record 3 of Set 1 | katherine
Processing Record 4 of Set 1 | bluff
Processing Record 5 of Set 1 | tasiilaq
Processing Record 6 of Set 1 | sijunjung
Processing Record 7 of Set 1 | thompson
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | hilo
Processing Record 10 of Set 1 | bredasdorp
Processing Record 11 of Set 1 | pacifica
Processing Record 12 of Set 1 | mataura
Processing Record 13 of Set 1 | shar
Processing Record 14 of Set 1 | hithadhoo
Processing Record 15 of Set 1 | faanui
Processing Record 16 of Set 1 | aksu
Processing Record 17 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 18 of Set 1 | hermanus
Processing Record 19 of Set 1 | the valley
Processing Record 20 of Set 1 | coahuayana
Processing Record 21 of Set 1 | arraial do cabo
Processing Record 22 of Set 1 | victoria
Processing Rec

Processing Record 37 of Set 4 | tra vinh
Processing Record 38 of Set 4 | tevriz
Processing Record 39 of Set 4 | netrakona
Processing Record 40 of Set 4 | mersing
Processing Record 41 of Set 4 | suez
Processing Record 42 of Set 4 | tungkang
City not found. Skipping...
Processing Record 43 of Set 4 | dikson
Processing Record 44 of Set 4 | mpika
Processing Record 45 of Set 4 | fort nelson
Processing Record 46 of Set 4 | qaanaaq
Processing Record 47 of Set 4 | solnechnyy
Processing Record 48 of Set 4 | olafsvik
Processing Record 49 of Set 4 | kribi
Processing Record 50 of Set 4 | lumphat
Processing Record 1 of Set 5 | edd
Processing Record 2 of Set 5 | icara
Processing Record 3 of Set 5 | matara
Processing Record 4 of Set 5 | esperance
Processing Record 5 of Set 5 | morris
Processing Record 6 of Set 5 | peniche
Processing Record 7 of Set 5 | sorland
Processing Record 8 of Set 5 | kalush
Processing Record 9 of Set 5 | gravdal
Processing Record 10 of Set 5 | carnarvon
Processing Record 11 of

Processing Record 29 of Set 8 | bambous virieux
Processing Record 30 of Set 8 | dingle
Processing Record 31 of Set 8 | port hardy
Processing Record 32 of Set 8 | waitati
Processing Record 33 of Set 8 | tessaoua
Processing Record 34 of Set 8 | jiayuguan
Processing Record 35 of Set 8 | baykit
Processing Record 36 of Set 8 | korla
Processing Record 37 of Set 8 | pisz
Processing Record 38 of Set 8 | mayo
Processing Record 39 of Set 8 | bacuit
City not found. Skipping...
Processing Record 40 of Set 8 | churapcha
Processing Record 41 of Set 8 | lac-megantic
Processing Record 42 of Set 8 | sinnamary
Processing Record 43 of Set 8 | port blair
Processing Record 44 of Set 8 | emerald
Processing Record 45 of Set 8 | tarazona
Processing Record 46 of Set 8 | angoche
Processing Record 47 of Set 8 | lorengau
Processing Record 48 of Set 8 | bakel
Processing Record 49 of Set 8 | zastron
Processing Record 50 of Set 8 | glendive
Processing Record 1 of Set 9 | cockburn town
Processing Record 2 of Set 9 | 

Processing Record 13 of Set 12 | narsaq
Processing Record 14 of Set 12 | sakakah
Processing Record 15 of Set 12 | chapais
Processing Record 16 of Set 12 | okhotsk
Processing Record 17 of Set 12 | alice town
Processing Record 18 of Set 12 | kamwenge
Processing Record 19 of Set 12 | cordoba
Processing Record 20 of Set 12 | tsihombe
City not found. Skipping...
Processing Record 21 of Set 12 | northam
Processing Record 22 of Set 12 | smiltene
Processing Record 23 of Set 12 | bridgetown
Processing Record 24 of Set 12 | kodinsk
Processing Record 25 of Set 12 | sigerfjord
City not found. Skipping...
Processing Record 26 of Set 12 | bol
Processing Record 27 of Set 12 | santiago del estero
Processing Record 28 of Set 12 | uong bi
Processing Record 29 of Set 12 | nelson bay
Processing Record 30 of Set 12 | omsukchan
Processing Record 31 of Set 12 | arsikere
Processing Record 32 of Set 12 | ostend
Processing Record 33 of Set 12 | monrovia
Processing Record 34 of Set 12 | laguna
Processing Record 

Processing Record 50 of Set 15 | aklavik
Processing Record 1 of Set 16 | kalavad
Processing Record 2 of Set 16 | mattru
Processing Record 3 of Set 16 | jam sahib
Processing Record 4 of Set 16 | clyde
Processing Record 5 of Set 16 | kangaatsiaq
Processing Record 6 of Set 16 | orda
Processing Record 7 of Set 16 | cap-aux-meules
Processing Record 8 of Set 16 | strezhevoy
Processing Record 9 of Set 16 | imeni babushkina
Processing Record 10 of Set 16 | san bartolome de tirajana
Processing Record 11 of Set 16 | bose
Processing Record 12 of Set 16 | sabang
Processing Record 13 of Set 16 | bato
Processing Record 14 of Set 16 | hrodna
Processing Record 15 of Set 16 | jaciara
Processing Record 16 of Set 16 | vung tau
Processing Record 17 of Set 16 | seydi
Processing Record 18 of Set 16 | meckenheim
Processing Record 19 of Set 16 | wasilla
Processing Record 20 of Set 16 | doha
Processing Record 21 of Set 16 | komsomolskiy
Processing Record 22 of Set 16 | cam ranh
Processing Record 23 of Set 16 |

In [37]:
len(city_data)

741

In [38]:
# Convert the array of dictionaries to a Pandas DataFrame.

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Saint-Philippe,RE,-21.3585,55.7679,71.19,78,31,13.53,light rain
1,Kavaratti,IN,10.5669,72.6420,81.19,77,100,17.60,overcast clouds
2,Katherine,AU,-14.4667,132.2667,62.92,39,2,6.91,clear sky
3,Bluff,NZ,-46.6000,168.3333,46.80,79,31,6.22,scattered clouds
4,Tasiilaq,GL,65.6145,-37.6368,41.16,78,100,2.42,overcast clouds
5,Sijunjung,ID,-0.6876,100.9537,73.51,97,98,1.39,overcast clouds
6,Thompson,CA,55.7435,-97.8558,61.05,71,75,5.91,broken clouds
7,Ushuaia,AR,-54.8000,-68.3000,37.02,100,100,16.11,light snow
8,Hilo,US,19.7297,-155.0900,76.28,80,100,5.75,overcast clouds
9,Bredasdorp,ZA,-34.5322,20.0403,60.82,49,7,16.62,clear sky


In [39]:
# Create the output file (CSV).

output_data_file = "weather_database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.

city_data_df.to_csv(output_data_file, index_label="City_ID")